# Exploratory Data Analysis
### 02/17/2020
### Group 6: Lukas Hering, Manothay Tommy Luangrath, Ian Luck, Andrea Simenstad

# Overview

<img src="used-car.jpg">

In this project, we are presented with some data and a business problem. Our goal is to provide a solution to our business problem while using the spark framework.


#### The Dataset

The dataset that we are provided with is the "Used Cars Database" scraped from Ebay Kleinanzeigen in Germany. The dataset contains a collection of over 370,000 used cars with a list of fields. The fields for our dataset include dateCrawled, name of the care, sellect, model of the car, kilometers the car has driven etc.

#### Business Problem


Our business problem is to predict how much a used car should be valued at, given relavent information such as the name of the car, the type of seller, offer type, how many kilometers the car has driven, etc. 

#### Main Object

The main objective for this project, is to build an efficient and effective model that is able to predict the sale price of a used vehicle. With our model, we will able to provide eBay or other car dealerships an efficient algorithm to predict the price a used car should be sold at in order to maximize the profit and to minimize time spent evaluating the price of a vehicle.

To achieve our main objective, we will be creating an ensemble model consisting multiple types of linnear regression models includeing Lasso regression, ridge regression and elastic net regression. 



# Exploratory Data Analysis

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col

spark = SparkSession.builder.master("local[*]").appName("Learning_Spark").getOrCreate()

First, we're going to load our data in our csv file into a spark dataframe in order to used our data with Spark.

In [2]:
data = spark.read.csv('autos.csv',inferSchema=True, header=True)

Now, we'll look at the shape of our dataframe to how many features and how much data we have.

In [4]:
print("Number of entries: " + str(data.count()))
print("Number of features: " + str(len(data.columns)))

Number of entries: 371824
Number of features: 20


Now we can print a few portions of our dataset to get an indication of what our data contains.

In [5]:
data.show(3)

+-------------------+--------------------+------+---------+-----+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|        dateCrawled|                name|seller|offerType|price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|        dateCreated|nrOfPictures|postalCode|           lastSeen|
+-------------------+--------------------+------+---------+-----+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|2016-03-24 11:52:17|          Golf_3_1.6|privat|  Angebot|  480|  test|       null|              1993|  manuell|      0| golf|   150000|                  0|  benzin|volkswagen|             null|2016-03-24 00:00:00|     

In [8]:
data.printSchema()

root
 |-- dateCrawled: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: timestamp (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- lastSeen: timestamp (nullable = true)



In [6]:
data.select("name","dateCreated","lastSeen","price").show(15, truncate=False)

+-----------------------------------------------------------------+-------------------+-------------------+-----+
|name                                                             |dateCreated        |lastSeen           |price|
+-----------------------------------------------------------------+-------------------+-------------------+-----+
|Golf_3_1.6                                                       |2016-03-24 00:00:00|2016-04-07 03:16:57|480  |
|A5_Sportback_2.7_Tdi                                             |2016-03-24 00:00:00|2016-04-07 01:46:50|18300|
|Jeep_Grand_Cherokee_"Overland"                                   |2016-03-14 00:00:00|2016-04-05 12:47:46|9800 |
|GOLF_4_1_4__3T�RER                                               |2016-03-17 00:00:00|2016-03-17 17:40:17|1500 |
|Skoda_Fabia_1.4_TDI_PD_Classic                                   |2016-03-31 00:00:00|2016-04-06 10:17:21|3600 |
|BMW_316i___e36_Limousine___Bastlerfahrzeug__Export               |2016-04-04 00:00:00|2

In [7]:
data.describe(["price","kilometer"]).show()

+-------+------------------+------------------+
|summary|             price|         kilometer|
+-------+------------------+------------------+
|  count|            371823|            371823|
|   mean|17286.338865535483|125618.56044408226|
| stddev|3586530.1840678076| 40111.62016494461|
|    min|                 0|              5000|
|    max|        2147483647|            150000|
+-------+------------------+------------------+



In [8]:
group_by_gearbox = data.groupby('gearbox')
group_by_brand = data.groupby('brand')
group_by_notRepairedDamage = data.groupby('notRepairedDamage')

In [9]:
group_by_gearbox.agg({'price': 'avg'}).show()

+---------+------------------+
|  gearbox|        avg(price)|
+---------+------------------+
|     null|43990.346585570886|
|automatik|15145.544156332206|
|  manuell|15920.486752589904|
+---------+------------------+



In [10]:
group_by_brand.agg({'price': 'avg'}).show()

+-------------+------------------+
|        brand|        avg(price)|
+-------------+------------------+
|       jaguar|14228.083735909822|
|     daihatsu|1691.6815365551424|
|   mitsubishi|3274.6647078028077|
|         null|              null|
|         lada|3037.4444444444443|
|       toyota| 5235.104468085106|
|         seat| 4356.156703672075|
|         saab| 3808.686090225564|
|   land_rover|16698.858625162127|
|      peugeot| 3168.667391698387|
|     chrysler| 3912.255158184319|
|      citroen| 8875.202082529888|
|         audi| 15863.35696264097|
|mercedes_benz|17244.502687715725|
|          bmw|14838.656460137465|
|         jeep|11998.898514851486|
|       lancia| 3165.754132231405|
|        skoda| 6413.099698955198|
|        rover| 1508.022448979592|
|      hyundai| 5417.859139490271|
+-------------+------------------+
only showing top 20 rows



In [11]:
group_by_brand.agg({'price': 'count'}).show()

+-------------+------------+
|        brand|count(price)|
+-------------+------------+
|       jaguar|         621|
|     daihatsu|         807|
|   mitsubishi|        3063|
|         null|           0|
|         lada|         225|
|       toyota|        4700|
|         seat|        7026|
|         saab|         532|
|   land_rover|         771|
|      peugeot|       11034|
|     chrysler|        1454|
|      citroen|        5186|
|         audi|       32897|
|mercedes_benz|       35346|
|          bmw|       40301|
|         jeep|         808|
|       lancia|         484|
|        skoda|        5647|
|        rover|         490|
|      hyundai|        3649|
+-------------+------------+
only showing top 20 rows



In [12]:
group_by_notRepairedDamage.agg({'price': 'avg'}).show()

+-----------------+-----------------+
|notRepairedDamage|       avg(price)|
+-----------------+-----------------+
|             nein|9128.952507688218|
|             null| 22760.0665252416|
|               ja|65586.87130267144|
+-----------------+-----------------+



In [13]:
group_by_notRepairedDamage.agg({'price': 'count'}).show()

+-----------------+------------+
|notRepairedDamage|count(price)|
+-----------------+------------+
|             nein|      263390|
|             null|       72123|
|               ja|       36310|
+-----------------+------------+



Here we see that many of our fields are Strings so we will need to convert some of these fields into more presentable formats. 

In [22]:
for col in data.columns:
    print(col, "\t", "number of null: ", data.filter(data[col].isNull()).count())

dateCrawled 	 number of null:  0
name 	 number of null:  0
seller 	 number of null:  1
offerType 	 number of null:  1
price 	 number of null:  1
abtest 	 number of null:  1
vehicleType 	 number of null:  37900
yearOfRegistration 	 number of null:  1
gearbox 	 number of null:  20224
powerPS 	 number of null:  1
model 	 number of null:  20499
kilometer 	 number of null:  1
monthOfRegistration 	 number of null:  1
fuelType 	 number of null:  33416
brand 	 number of null:  1
notRepairedDamage 	 number of null:  72124
dateCreated 	 number of null:  1
nrOfPictures 	 number of null:  1
postalCode 	 number of null:  1
lastSeen 	 number of null:  1


As we can see, we have quite a few null values. Now we will have to fill these null values. 